In [1]:
import pickle
import re

import random
import numpy as np
from collections import Counter
from keras.models import Model
from keras.layers import Input, Dense, GRU, Dropout
from matplotlib import pyplot

Using TensorFlow backend.


In [8]:
import sys
sys.path.insert(0, '../src/')
import kernel_util
import discretize_f0

In [4]:
### Load data ###
chords_midis_beats = pickle.load(open("../src/chords-midis-beats.pkl", 'rb'))
chords_f0s_beats = pickle.load(open("../src/chords-f0s-beats.pkl", 'rb'))

midi_chords = [part["chords"] for sid, song in chords_midis_beats.items() for part in song]
midi_midis = [part["midis"] for sid, song in chords_midis_beats.items() for part in song]
midi_beats = [part["beats"] for sid, song in chords_midis_beats.items() for part in song]

f0_chords = [part["chords"] for sid, song in chords_f0s_beats.items() for part in song]
f0_f0s = [part["f0s"] for sid, song in chords_f0s_beats.items() for part in song]
f0_beats = [part["beats"] for sid, song in chords_f0s_beats.items() for part in song]
# print(set([chord for part in chords for chord in part]))

In [5]:
### Train/Test splitting ###
indices = [145, 92, 112, 161, 24, 266, 21, 83, 308, 246, 326, 126, 332, 65, 247, 263, 294, 68, 313, 156, 143, 344, 62, 210, 39, 84, 340, 134, 274, 257, 167, 48, 299, 102, 11, 331, 181, 214, 18, 90, 225, 82, 286, 56, 71, 153, 251, 256, 237, 287, 80, 137, 14, 131, 115, 309, 254, 273, 6, 307, 269, 315, 87, 19, 22, 125, 49, 305, 2, 133, 136, 250, 335, 334, 229, 105, 146, 47, 185, 107, 228, 261, 338, 322, 330, 123, 12, 94, 191, 292, 76, 135, 304, 122, 204, 333, 194, 291, 317, 60, 152, 61, 179, 173, 158, 280, 13, 336, 283, 195, 244, 85, 1, 8, 242, 119, 45, 255, 114, 279, 4, 74, 116, 52, 36, 215, 290, 197, 132, 258, 140, 139, 276, 206, 319, 342, 63, 20, 186, 168, 328, 303, 144, 138, 16, 199, 155, 213, 311, 264, 183, 30, 130, 55, 7, 108, 109, 5, 234, 190, 148, 217, 44, 298, 159, 265, 323, 154, 272, 151, 252, 118, 288, 23, 70, 216, 104, 9, 3, 64, 42, 262, 172, 259, 128, 211, 142, 275, 184, 223, 96, 219, 327, 343, 178, 300, 171, 243, 226, 166, 17, 41, 165, 189, 238, 175, 25, 72, 86, 35, 124, 37, 15, 177, 32, 297, 203, 77, 278, 33, 75, 157, 170, 147, 117, 34, 236, 120, 193, 221, 341, 73, 51, 57, 78, 110, 27, 224, 188, 282, 182, 271, 29, 239, 222, 40, 232, 296, 43, 218, 100, 293, 312, 201, 241, 150, 200, 233, 248, 281, 306, 97, 98, 28, 160, 10, 88, 207, 249, 66, 277, 240, 270, 95, 337, 163, 314, 99, 231, 162, 329, 38, 176, 267, 316, 198, 192, 202, 245, 196, 268, 111, 129, 0, 285, 164, 227, 209, 324, 54, 212, 289, 59, 31, 89, 220, 113, 301, 339, 208, 106, 180, 81, 67, 53, 79, 69, 46, 318, 310, 169, 127, 230, 50, 141, 26, 302, 260, 103, 187, 321, 295, 91, 149, 101, 205, 320, 93, 253, 325, 174, 121, 235, 284, 58]

train_indices = indices[0:round(len(midi_chords)*0.75)]
test_indices = indices[round(len(midi_chords)*0.75):]

# data for midi
midi_chords_train = [midi_chords[i] for i in train_indices]
midi_chords_test = [midi_chords[i] for i in test_indices]
midi_midis_train = [midi_midis[i] for i in train_indices]
midi_midis_test = [midi_midis[i] for i in test_indices]
midi_beats_train = [midi_beats[i] for i in train_indices]
midi_beats_test = [midi_beats[i] for i in test_indices]

f0_chords_train = [f0_chords[i] for i in train_indices]
f0_chords_test = [f0_chords[i] for i in test_indices]
f0_f0s_train = [f0_f0s[i] for i in train_indices]
f0_f0s_test = [f0_f0s[i] for i in test_indices]
f0_beats_train = [f0_beats[i] for i in train_indices]
f0_beats_test = [f0_beats[i] for i in test_indices]

In [13]:
### Preprocess data ###

# Train data (from MIDI data)
beat_midi_chords_train = []
beat_midi_midis_train = []
for (part_c, part_m, part_b) in zip(midi_chords_train, midi_midis_train, midi_beats_train):
    part_m = discretize_f0.to_octave_range(part_m) # convert midi data to octave range
    part_midi_seqs, part_chords = kernel_util.split_in_beats(part_c, part_m, part_b)
    beat_midi_chords_train += [kernel_util.truncate_chord_to_triad_rwc(chord) for chord in part_chords]
    beat_midi_midis_train += part_midi_seqs
    
# Test data (from F0 data)
beat_f0_chords_test = []
beat_f0_f0s_test = []
for (part_c, part_f, part_b) in zip(f0_chords_test, f0_f0s_test, f0_beats_test):
    part_f = discretize_f0.to_octave_range(discretize_f0.discretize(part_f)) # discretize F0 data and bring to octave range
    part_f0_seqs, part_chords = kernel_util.split_in_beats(part_c, part_f, part_b)
    beat_f0_chords_test += [kernel_util.truncate_chord_to_triad_rwc(chord) for chord in part_chords]
    beat_f0_f0s_test += part_f0_seqs

In [14]:
### Target classes mapping ###
total_chords = []
for part in f0_chords:
    for chord in part:
        total_chords.append(kernel_util.truncate_chord_to_triad_rwc(chord))

distinct_target = list(set(total_chords))
chords_label_index = dict(
    [(c, i) for i, c in enumerate(sorted(distinct_target))])

mapping_to_chord = {}
for k, v in chords_label_index.items():
    mapping_to_chord[v] = k

In [15]:
### Variables definition ###
num_samples_train = len(beat_midi_midis_train)
num_samples_test = len(beat_f0_f0s_test)
max_input_seq_len = max([len(beat_seq) for beat_seq in beat_midi_midis_train+beat_f0_f0s_test])
input_vector_size = 1
output_vector_size = len(chords_label_index)
num_input_classes = 13

print("Total number of input train samples:", num_samples_train)
print("Total number of input test samples:", num_samples_test)
print("Maximum input sequence length:", max_input_seq_len)
print("Vector size in each input sequence:", input_vector_size)
print("Total number of target chord labels:", output_vector_size)
print("Total number of input classes:", num_input_classes)

Total number of input train samples: 20510
Total number of input test samples: 7002
Maximum input sequence length: 97
Vector size in each input sequence: 1
Total number of target chord labels: 58
Total number of input classes: 13


In [17]:
### Build tensors ###

# Initialize tensors
X_train = np.zeros(
    (num_samples_train, max_input_seq_len, num_input_classes), # add two rows for the padding class and for the duration metric
    dtype='float32')

y_train = np.zeros(
    (num_samples_train, output_vector_size),
    dtype='float32')

X_test = np.zeros(
    (num_samples_test, max_input_seq_len, num_input_classes), # add two rows for the padding class and for the duration metric
    dtype='float32')

y_test = np.zeros(
    (num_samples_test, output_vector_size),
    dtype='float32')


# Fill tensors
for i, (beat_m, beat_c) in enumerate(zip(beat_midi_midis_train, beat_midi_chords_train)):
    y_train[i, chords_label_index[beat_c]] = 1    
    for j, m in enumerate(beat_m):
        X_train[i, j, m+1] = 1
        

for i, (beat_m, beat_c) in enumerate(zip(beat_f0_f0s_test, beat_f0_chords_test)):
    y_test[i, chords_label_index[beat_c]] = 1    
    for j, m in enumerate(beat_m):
        X_test[i, j, m+1] = 1

In [18]:
### Build model ###
hidden_units = 64

x = Input(shape=(max_input_seq_len, num_input_classes))
layer_1 = GRU(hidden_units, return_sequences=True)(x)
dropout_1 = Dropout(0.2)(layer_1)
layer_2 = GRU(hidden_units)(dropout_1)
dropout_2 = Dropout(0.2)(layer_2)
chord_prediction = Dense(output_vector_size, activation='softmax')(dropout_2)

model = Model(x, chord_prediction)
model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

In [ ]:
### Train models ###
batch_size = 32
epochs = 50

history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs, batch_size=batch_size)

In [ ]:
### Visualize learning ###
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['acc'])
pyplot.plot(history.history['val_acc'])
pyplot.title('model train vs validation acc')
pyplot.ylabel('acc')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
### Visualize generation ###

# Randomly pick song part
sample_index = random.randint(0, len(test_indices))
sample_ch, sample_f0, sample_bt = midi_chords_test[sample_index], midi_midis_test[sample_index], midi_beats_test[sample_index]
sample_f0_seqs, sample_chords = kernel_util.split_in_beats(sample_ch, sample_f0, sample_bt)
sample_chords = [kernel_util.truncate_chord_to_triad_rwc(chord) for chord in sample_chords]

# Build predict input tensor
n_beats = len(sample_f0_seqs)
X_predict = np.zeros(
    (n_beats, max_input_seq_len, input_vector_size),
    dtype='float32')
for i, b in enumerate(sample_f0_seqs):
    for j, s in enumerate(b):
        X_predict[i, j, 0] = s
        
# Predict and visualize output
y_predict = model.predict(X_predict, batch_size=1)

predicted_chords = []
for row in y_predict:
    index = np.argmax(row)
    predicted_chords.append(mapping_to_chord[index])
print(predicted_chords)